In [1]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.configurations.params import DEFAULT_STUDY_TABLES_LIST
from mimic_iv_analysis.io import DataLoader, TableNames, ParquetConverter, ExampleDataLoader
import pandas as pd

import dask.dataframe as dd

# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='8GB')
# client

In [ ]:
# example = ExampleDataLoader(partial_loading=False, num_subjects=10, apply_filtering=True)
# df = example.load_merged_tables()

In [2]:
loader = DataLoader()


In [21]:
df = loader.load_one_table(table_name=TableNames.ADMISSIONS, partial_loading=True, num_subjects=10)

[21:38:00] INFO     Filtering admissions by subject_id for 10 subjects.                          data_loader.py:393

## Scan the directory

In [3]:
df_merged = loader.load_merged_tables(partial_loading=False)

[21:53:01] INFO     Loaded 2423918 rows from merged table.                                       data_loader.py:525

In [4]:
subject_ids = loader.all_subject_ids(df=df_merged, table_name=TableNames.MERGED)

In [5]:
len(set(subject_ids))

4943

In [6]:
df_merged.shape[0].compute()

2423918